In [1]:
import sys
!{sys.executable} -m pip install faiss-cpu sentence-transformers langchain-text-splitters pandas numpy

In [2]:
import sys
!{sys.executable} -m pip install langchain-text-splitters

In [3]:
import sys
import os

# Adds the project root to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

In [4]:
import pandas as pd
import numpy as np

In [5]:
from src.config import *
from src.sampling import stratified_sample
from src.chunking import chunk_text
from src.embeddings import load_embedding_model
from src.vector_store import create_faiss_index
from src.utils import validate_dataframe, stack_embeddings


c:\Users\bia\Desktop\week-7\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [6]:
# Load data
import pandas as pd

# Load data
df = pd.read_csv("../data/processed/filtered_complaints.csv")

In [7]:
validate_dataframe(df, [TEXT_COLUMN, ID_COLUMN, STRATIFY_COLUMN])

In [8]:
# Stratified sampling
sampled_df = stratified_sample(
    df,
    SAMPLE_SIZE,
    STRATIFY_COLUMN,
    RANDOM_SEED
)

In [ ]:
sampled_df.to_csv("../data/sampled_complaints.csv", index=False)

In [10]:
import numpy as np

# Load model
model = load_embedding_model(EMBEDDING_MODEL_NAME)

all_embeddings = []
all_metadata = []

for _, row in sampled_df.iterrows():
    chunks = chunk_text(row[TEXT_COLUMN], CHUNK_SIZE, CHUNK_OVERLAP)

    for chunk in chunks:
        emb = model.encode(chunk)
        all_embeddings.append(emb)
        all_metadata.append({
            "complaint_id": row[ID_COLUMN],
            "product": row[STRATIFY_COLUMN],
            "text": chunk
        })

        embedding_matrix = stack_embeddings(all_embeddings)

In [11]:
# Create FAISS index
index = create_faiss_index(
    embedding_matrix,
    all_metadata,
    VECTOR_DB_PATH
)

print("✨ Vector store created and saved successfully.")

✨ Vector store created and saved successfully.
